# Collect the Dataset

We use, for example, the dataset present in the article, but if you want to train a model in your dataset, you must use your dataset. Or, if you're going to use the dataset present in the article with another model, you only must change the model

In [1]:
!wget https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Classification/Dataset/RevisoesSoftware.json

--2021-04-23 14:35:03--  https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Classification/Dataset/RevisoesSoftware.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4475705 (4.3M) [text/plain]
Saving to: ‘RevisoesSoftware.json’

RevisoesSoftware.js 100%[===================>]   4.27M  22.8MB/s    in 0.2s    

2021-04-23 14:35:04 (22.8 MB/s) - ‘RevisoesSoftware.json’ saved [4475705/4475705]



In [2]:
import pandas as pd
import json

with open('RevisoesSoftware.json', 'r') as f:
  data = json.load(f)

df_complete = pd.DataFrame(data)

# Word-Embeddings

## Install

In [3]:
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 2.1MB 8.6MB/s 
     |████████████████████████████████| 1.2MB 39.0MB/s 
     |████████████████████████████████| 901kB 40.6MB/s 
     |████████████████████████████████| 3.3MB 38.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=4e702d7e4c36082912c0a864efe84549fd1dd912fbfe89ef260c1c207773753c
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers


## Import

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

## Variations of word embeddings and how to use them

In [5]:
def WordEmbeddings(texts, model):

  if type(texts) == pd.core.series.Series:
    sentences = texts.replace(['\t','\n','\r'], [' ',' ',' '], regex=True)
  else:
    sentences = texts
  
  sentence_embeddings = model.encode(list(sentences))

  return sentence_embeddings 

In [6]:
dic_word_emb = {
    'BERT' : SentenceTransformer('bert-large-nli-stsb-mean-tokens'),
    'RoBERTa' : SentenceTransformer('roberta-large-nli-stsb-mean-tokens'),
    'DistilBERT' : SentenceTransformer('distilbert-base-nli-stsb-mean-tokens'),
    'DistilBERT ML' : SentenceTransformer('distiluse-base-multilingual-cased')
}

# Functions to train the Model



## import models

In [7]:
from sklearn.svm import OneClassSVM as OCSVM

## Define the algorithm that you will use

In [8]:
clf = OCSVM()

## Train-Test division

First, you must define a class of interest, second you must define the train set from the class of interest and the test set that contians examples from the class of interest and other classes. *test_size* define the percent of examples of test set, consequently, the train set size is 1 - *test_size*

In [9]:
from sklearn.model_selection import train_test_split

class_interest = 'Rating'

df_train_interest, df_test_interest = train_test_split(df_complete['comment'][df_complete['label'] == class_interest],test_size=0.25, random_state=42)
df_test_outliers = df_complete['comment'][df_complete['label'] != class_interest]

# Execution

## Pre-processing

In [10]:
x_train = WordEmbeddings(df_train_interest, dic_word_emb['BERT']) 

x_test_interest =  WordEmbeddings(df_test_interest,dic_word_emb['BERT']) 

x_test_outlier = WordEmbeddings(df_test_outliers,dic_word_emb['BERT']) 

## Train

In [12]:
clf.fit(x_train)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
            max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=False)

### Saving the model

In [ ]:
import pickle

pkl_filename = "pickle_OCSVM_BERT.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file) 

if you want to load the model, use:

with open(pkl_filename, 'rb') as file: \\
    clf = pickle.load(file)

## Test

In [13]:
y_pred_int = clf.predict(x_test_interest)
y_pred_out = clf.predict(x_test_outlier)

In [14]:
from sklearn.metrics import classification_report

def evaluation_one_class(preds_interest, preds_outliers):
  y_true = [1]*len(preds_interest) + [-1]*len(preds_outliers)
  y_pred = list(preds_interest)+list(preds_outliers)
  return classification_report(y_true, y_pred, output_dict=False)

In [15]:
print(evaluation_one_class(y_pred_int, y_pred_out))

              precision    recall  f1-score   support

          -1       0.75      0.75      0.75      1229
           1       0.50      0.50      0.50       616

    accuracy                           0.66      1845
   macro avg       0.62      0.62      0.62      1845
weighted avg       0.66      0.66      0.66      1845



# Study Case

In [16]:
texts = ['the app always crashes !!!!!!!!!!', 'I loved this app!!']

In [20]:
def Classification(text):
  embeddings_test = WordEmbeddings([text], dic_word_emb['BERT'])
  resp = clf.predict(embeddings_test)
  if resp[0] == 1:
    print('The text: "' + text + '" BELONGS to the class of interest!') 
  if resp[0] == -1:
    print('The text: "' + text + '" DOES NOT belong to the class of interest!') 

In [21]:
for text in texts:
  Classification(text)

The text: "the app always crashes !!!!!!!!!!" DOES NOT belong to the class of interest!
The text: "I loved this app!!" BELONGS to the class of interest!
